#### CMAPSS DATA (FD0001) 대상으로 데이터셋 구성
- 라벨(이상, 정상)
- time series dataset
- train 데이터만 사용, test데이터는 한 cycle이 끝나면 고장이라는 가정을 세울 수 없는 데이터임

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
cmapss_train_files = [#'./data/CMAPSSData/train_FD001.txt',
                      #'./data/CMAPSSData/train_FD002.txt',
                      #'./data/CMAPSSData/train_FD003.txt',
                      './data/CMAPSSData/train_FD004.txt'
                     ]

In [3]:
# label 달기 함수
def create_label(data, cycle_cnt, file_lable_cnt, padding):
    """
    data : 데이터
    cycle_cnt :  한 file에 cycle들이 몇개 있는가
    file_lable_cnt : 한 cycle에 'Y' 개수
    """
    label = []
    cycle_list = []    # cycle 수
    for cycle_num in range(1, cycle_cnt+1):
        idx_len = len(data[data[0]== cycle_num].index)
        cycle_list.append(idx_len)
        [label.append(0) if x < (idx_len - file_lable_cnt) else label.append(1)  for x in range(0,idx_len)]
        """
        for x in range(0, idx_len):
            if x < (idx_len - file_lable_cnt - padding): label.append(0)
            elif (idx_len - file_lable_cnt - padding) <= x and x < (idx_len - file_lable_cnt): label.append(1)
            else: label.append(2)
        """
    return cycle_list, label

In [5]:
# 각 cycle의 마지막  label_cnt 개수만큼의 row의 label을 'Y' 이상으로 지정 
# 한 파일마다 train test cycle을 splite_ratio로 나눔
train = pd.DataFrame()
test = pd.DataFrame()
splite_ratio = 0.8
label_cnt = 50
padding = 0    # 고장 라벨 전 단계(1) 갯수

for file_name in cmapss_train_files:
    # 라벨 생성
    data = pd.read_csv(file_name, sep = ' ', header = None)
    data = data.iloc[:,:-2]
    
    cycle_cnt = len(data.iloc[:,0].unique())
    splite_cycle_num = np.int(np.round(cycle_cnt*splite_ratio,0))   # train_test_splite ratio = 0.8
    
    cycle_list, label = create_label(data, cycle_cnt, label_cnt, padding)
    data['label'] = label
        
    # train / test splite
    # -- random 하게 cycle 숫자 생성
    trian_cycle_num = sorted(np.random.choice(range(1,cycle_cnt+1), splite_cycle_num, replace=False))
    
    #-- random하게 생성한 cycle 숫자들의 index추출
    trian_idx = []
    [trian_idx.extend(list(data[data[0] == cycle_num].index)) for cycle_num in trian_cycle_num  ]
    test_idx = data.index.difference(trian_idx)
    
    #-- index로 filtering
    train = train.append(data.iloc[trian_idx,:],ignore_index = True)
    test = test.append(data.iloc[test_idx,:],ignore_index = True)

    print('file name : {},  cnt : {}, splite_num : {} '.format(file_name,cycle_cnt,splite_cycle_num))
    print('file name : {},  train_unique : {}, test_unique : {} '.format(file_name,len(train[0].unique()),len(test[0].unique())))
    print('file name : {},  train : {}, test : {} '.format(file_name,train.shape[0],test.shape[0]))

file name : ./data/CMAPSSData/train_FD004.txt,  cnt : 249, splite_num : 199 
file name : ./data/CMAPSSData/train_FD004.txt,  train_unique : 199, test_unique : 50 
file name : ./data/CMAPSSData/train_FD004.txt,  train : 49807, test : 11442 


In [6]:

train.to_csv('C:/Users/ARA/Desktop/금형/python/data/train_failure50_4.csv',
                    index = False, encoding = 'cp949')
test.to_csv('C:/Users/ARA/Desktop/금형/python/data/test_failure50_4.csv',
                    index = False, encoding = 'cp949')


# time series dataset
- 중심데이터는 데이터pack의 가장 최신 데이터 /n
- 데이터 pack의 라벨은 중심 데이터의 다음 데이터의 라벨

- pack : 고려할 과거 데이터 pack (row 묶음)
- stride : 데이터pack 간의 간격
- 각 엔진의 마지막 데이터 pack의 라벨은 'Y'

In [49]:
# unique columns 추출
uniq_col = []
for i in range(train.shape[1]):
    if len(np.unique(train.iloc[:,i], axis=0)) != 1: uniq_col.append(i)

In [50]:
uniq_col.remove(0)
uniq_col.remove(1)
uniq_col.remove(26)

In [51]:
train.rename(columns={0:'id', 1:'cycle'}, inplace=True)

In [52]:
train.head()

,id,cycle,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,label
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,0
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,0
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,0
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,0
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,0


In [53]:
# scaling
sc = StandardScaler()
train[uniq_col] = sc.fit_transform(train[uniq_col])
#test[uniq_col] = sc.fit_transform(test[uniq_col])

C:\Users\ARA\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\ARA\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [54]:
seq_length = 100 #window size
stride = 2
seq_cols = uniq_col

In [55]:
# sequence matrix 생성
def make_sequence_matrix(X, seq_cols=seq_cols, seq_length=seq_length, stride=stride):
    X = X.reset_index(drop=True)
    data_array = X[seq_cols].values
    num_elements = data_array.shape[0]
    sequence_array = []
    for start, stop in zip(range(0, num_elements - seq_length+1, stride), range(seq_length, num_elements+1, stride)):
        #if start < 5: continue    #초반 5개 제거
        if start > 30 and X['label'][stop-1]==0: continue
        sequence_array.append(data_array[start:stop,:])

    return np.array(sequence_array)

In [56]:
# sequence matrix label 생성
def make_sequence_label(X, label, seq_col=seq_cols, seq_length=seq_length, stride=stride):
    X = X.reset_index(drop=True)
    data_array = X[seq_cols].values
    num_elements = data_array.shape[0]
    #print(num_elements)
    y_label = []
    for start, stop in zip(range(0, num_elements - seq_length+1, stride), range(seq_length, num_elements+1, stride)):
        #if start < 5: continue    #초반 5개 제거
        if start > 30 and X[label][stop-1]==0: continue
        y_label.append(X[label][stop-1])
        print(start, stop, X[label][stop-1], len(y_label))

    return np.array(y_label)

In [57]:
train_seq = np.concatenate(list(list(make_sequence_matrix(train[train['id']==id])) for id in train['id'].unique()))
train_seq_label = np.concatenate(list(list(make_sequence_label(train[train['id']==id], 'label')) for id in train['id'].unique()))
print("train_seq.shape: ", train_seq.shape)
print("train_seq_label.shape: ", train_seq_label.shape)

0 100 0 1
2 102 0 2
4 104 0 3
6 106 0 4
8 108 0 5
10 110 0 6
12 112 0 7
14 114 0 8
16 116 0 9
18 118 0 10
20 120 0 11
22 122 0 12
24 124 0 13
26 126 0 14
28 128 0 15
30 130 0 16
64 164 1 17
66 166 1 18
68 168 1 19
70 170 1 20
72 172 1 21
74 174 1 22
76 176 1 23
78 178 1 24
80 180 1 25
82 182 1 26
84 184 1 27
86 186 1 28
88 188 1 29
90 190 1 30
92 192 1 31
0 100 0 1
2 102 0 2
4 104 0 3
6 106 0 4
8 108 0 5
10 110 0 6
12 112 0 7
14 114 0 8
16 116 0 9
18 118 0 10
20 120 0 11
22 122 0 12
24 124 0 13
26 126 0 14
28 128 0 15
30 130 0 16
158 258 1 17
160 260 1 18
162 262 1 19
164 264 1 20
166 266 1 21
168 268 1 22
170 270 1 23
172 272 1 24
174 274 1 25
176 276 1 26
178 278 1 27
180 280 1 28
182 282 1 29
184 284 1 30
186 286 1 31
0 100 0 1
2 102 0 2
4 104 0 3
6 106 0 4
8 108 0 5
10 110 0 6
12 112 0 7
14 114 0 8
16 116 0 9
18 118 0 10
20 120 0 11
22 122 0 12
24 124 0 13
26 126 0 14
28 128 0 15
30 130 0 16
50 150 1 17
52 152 1 18
54 154 1 19
56 156 1 20
58 158 1 21
60 160 1 22
62 162 1 23
64 164 

40 140 1 21
42 142 1 22
44 144 1 23
46 146 1 24
0 100 0 1
2 102 0 2
4 104 0 3
6 106 0 4
8 108 0 5
10 110 0 6
12 112 0 7
14 114 0 8
16 116 0 9
18 118 0 10
20 120 0 11
22 122 0 12
24 124 0 13
26 126 0 14
28 128 0 15
30 130 0 16
102 202 1 17
104 204 1 18
106 206 1 19
108 208 1 20
110 210 1 21
112 212 1 22
114 214 1 23
116 216 1 24
118 218 1 25
120 220 1 26
122 222 1 27
124 224 1 28
126 226 1 29
128 228 1 30
130 230 1 31
0 100 0 1
2 102 0 2
4 104 0 3
6 106 0 4
8 108 0 5
10 110 0 6
12 112 0 7
14 114 0 8
16 116 0 9
18 118 0 10
20 120 0 11
22 122 0 12
24 124 0 13
26 126 0 14
28 128 0 15
30 130 0 16
70 170 1 17
72 172 1 18
74 174 1 19
76 176 1 20
78 178 1 21
80 180 1 22
82 182 1 23
84 184 1 24
86 186 1 25
88 188 1 26
90 190 1 27
92 192 1 28
94 194 1 29
96 196 1 30
98 198 1 31
0 100 0 1
2 102 0 2
4 104 0 3
6 106 0 4
8 108 0 5
10 110 0 6
12 112 0 7
14 114 0 8
16 116 0 9
18 118 0 10
20 120 0 11
22 122 0 12
24 124 0 13
26 126 0 14
28 128 1 15
30 130 1 16
32 132 1 17
34 134 1 18
36 136 1 19
38 138 

106 206 1 28
108 208 1 29
110 210 1 30
112 212 1 31
0 100 0 1
2 102 0 2
4 104 0 3
6 106 0 4
8 108 0 5
10 110 0 6
12 112 0 7
14 114 0 8
16 116 0 9
18 118 0 10
20 120 0 11
22 122 0 12
24 124 0 13
26 126 0 14
28 128 0 15
30 130 0 16
66 166 1 17
68 168 1 18
70 170 1 19
72 172 1 20
74 174 1 21
76 176 1 22
78 178 1 23
80 180 1 24
82 182 1 25
84 184 1 26
86 186 1 27
88 188 1 28
90 190 1 29
92 192 1 30
94 194 1 31
0 100 0 1
2 102 0 2
4 104 0 3
6 106 0 4
8 108 0 5
10 110 0 6
12 112 0 7
14 114 0 8
16 116 0 9
18 118 0 10
20 120 0 11
22 122 0 12
24 124 0 13
26 126 0 14
28 128 0 15
30 130 0 16
128 228 1 17
130 230 1 18
132 232 1 19
134 234 1 20
136 236 1 21
138 238 1 22
140 240 1 23
142 242 1 24
144 244 1 25
146 246 1 26
148 248 1 27
150 250 1 28
152 252 1 29
154 254 1 30
156 256 1 31
0 100 0 1
2 102 0 2
4 104 0 3
6 106 0 4
8 108 0 5
10 110 0 6
12 112 0 7
14 114 0 8
16 116 0 9
18 118 0 10
20 120 0 11
22 122 0 12
24 124 0 13
26 126 0 14
28 128 0 15
30 130 0 16
64 164 1 17
66 166 1 18
68 168 1 19
70 

0 100 0 1
2 102 0 2
4 104 0 3
6 106 0 4
8 108 0 5
10 110 0 6
12 112 0 7
14 114 0 8
16 116 0 9
18 118 0 10
20 120 0 11
22 122 0 12
24 124 0 13
26 126 0 14
28 128 0 15
30 130 0 16
164 264 1 17
166 266 1 18
168 268 1 19
170 270 1 20
172 272 1 21
174 274 1 22
176 276 1 23
178 278 1 24
180 280 1 25
182 282 1 26
184 284 1 27
186 286 1 28
188 288 1 29
190 290 1 30
192 292 1 31
0 100 0 1
2 102 0 2
4 104 0 3
6 106 0 4
8 108 0 5
10 110 0 6
12 112 0 7
14 114 0 8
16 116 0 9
18 118 0 10
20 120 0 11
22 122 0 12
24 124 0 13
26 126 0 14
28 128 0 15
30 130 0 16
138 238 1 17
140 240 1 18
142 242 1 19
144 244 1 20
146 246 1 21
148 248 1 22
150 250 1 23
152 252 1 24
154 254 1 25
156 256 1 26
158 258 1 27
160 260 1 28
162 262 1 29
164 264 1 30
166 266 1 31
0 100 0 1
2 102 0 2
4 104 0 3
6 106 0 4
8 108 0 5
10 110 0 6
12 112 0 7
14 114 0 8
16 116 0 9
18 118 0 10
20 120 0 11
22 122 0 12
24 124 0 13
26 126 0 14
28 128 0 15
30 130 0 16
60 160 1 17
62 162 1 18
64 164 1 19
66 166 1 20
68 168 1 21
70 170 1 22
72 1

In [58]:
test_seq, test_seq_label = make_sequence_matrix(test, seq_cols)
print("test_seq.shape: ", test_seq.shape)
print("test_seq_label.shape: ", test_seq_label.shape)

ValueError: not enough values to unpack (expected 2, got 0)

In [59]:
np.save('train_seq_30N_5out_mini_100w', train_seq)
np.save('train_seq_label_30N_5out_mini_100w', train_seq_label)
#np.save('test_seq', test_seq)
#np.save('test_seq_label', test_seq_label)

In [68]:
# sequence matrix 생성 -> LSTM을 위한 것으로, 앞에 0 패딩을 붙임
def gen_sequence(X, seq_cols, seq_length=seq_length, stride=stride):
    df_zeros = pd.DataFrame(np.zeros((seq_length-1, X.shape[1])), columns=X.columns)
    X = df_zeros.append(X, ignore_index=True)
    data_array = X[seq_cols].values
    num_elements = data_array.shape[0]
    lstm_array = []
    y_label = []
    for start, stop in zip(range(0, num_elements - seq_length, stride), range(seq_length, num_elements, stride)):
        lstm_array.append(data_array[start:stop,:])
        y_label.append(X['label'][stop])

    return np.array(lstm_array), np.array(y_label)

In [20]:
train_gen_seq, train_gen_seq_label = gen_sequence(train, seq_cols)
print("train_gen_seq.shape: ", train_gen_seq.shape)
print("train_gen_seq_label.shape: ", train_gen_seq_label.shape)

train_gen_seq.shape:  (16138, 50, 17)
train_gen_seq_label.shape:  (16138,)


In [21]:
test_gen_seq, test_gen_seq_label = gen_sequence(test, seq_cols)
print("test_gen_seq.shape: ", test_gen_seq.shape)
print("test_gen_seq_label.shape: ", test_gen_seq_label.shape)

test_gen_seq.shape:  (4491, 50, 17)
test_gen_seq_label.shape:  (4491,)


In [22]:
np.save('train_gen_seq', train_gen_seq)
np.save('train_gen_seq_label', train_gen_seq_label)
np.save('test_gen_seq', test_gen_seq)
np.save('test_gen_seq_label', test_gen_seq_label)